In [1]:
# Avec Pandas
import pandas as pd
import polars

df = pd.read_csv(
    "../data/raw/osm/poi.csv/poi.csv",
    sep="\t",
    engine="python",   # <--- ultra permissif
    on_bad_lines="skip"   # ou "warn"
)

# Mairie de Quimper
lat0 = 47.996197
lon0 = -4.102031

In [2]:
# osmium cat poi.csv -f csv > cleaned.csv
# Outil pour nettoyer
import polars as pl

print(df.head(5))

oms_df = pl.from_pandas(df)
df = None

      @id      @lon       @lat bBox                  name alt_name short_name  \
0  124585  5.697613  45.152329  NaN  Espace Comboire Nord      NaN        NaN   
1  126251  5.611789  45.302730  NaN                   NaN      NaN        NaN   
2  126264  5.587801  45.301981  NaN                   NaN      NaN        NaN   
3  126301  5.524429  45.286404  NaN               Tullins      NaN        NaN   
4  126358  5.424495  45.199277  NaN                 Vinay      NaN        NaN   

  name:fr local_name official_name  ... military mountain_pass natural office  \
0     NaN        NaN           NaN  ...      NaN           NaN     NaN    NaN   
1     NaN        NaN           NaN  ...      NaN           NaN     NaN    NaN   
2     NaN        NaN           NaN  ...      NaN           NaN     NaN    NaN   
3     NaN        NaN           NaN  ...      NaN           NaN     NaN    NaN   
4     NaN        NaN           NaN  ...      NaN           NaN     NaN    NaN   

  place railway shop touri

In [5]:
import math
import polars as pl

# osm_df92000 = oms_df.filter(pl.col("local_name") == "quimper")
osm_df92000 = oms_df.filter()

print(oms_df.head().to_dicts())

print(len(osm_df92000))


[{'@id': 124585, '@lon': 5.6976134, '@lat': 45.1523291, 'bBox': None, 'name': 'Espace Comboire Nord', 'alt_name': None, 'short_name': None, 'name:fr': None, 'local_name': None, 'official_name': None, 'old_name': None, 'operator': None, 'brand': None, 'ref': '6', 'aerialway': None, 'aerodrome': None, 'aeroway': None, 'amenity': None, 'boundary': None, 'bridge': None, 'craft': None, 'emergency': None, 'heritage': None, 'highway': 'motorway_junction', 'historic': None, 'junction': None, 'landuse': None, 'leisure': None, 'man_made': None, 'military': None, 'mountain_pass': None, 'natural': None, 'office': None, 'place': None, 'railway': None, 'shop': None, 'tourism': None, 'tunnel': None, 'waterway': None}, {'@id': 126251, '@lon': 5.6117888, '@lat': 45.3027298, 'bBox': None, 'name': None, 'alt_name': None, 'short_name': None, 'name:fr': None, 'local_name': None, 'official_name': None, 'old_name': None, 'operator': None, 'brand': None, 'ref': 'G1', 'aerialway': None, 'aerodrome': None, 'aer

In [3]:
import polars as pl
print(pl.__version__)


1.35.2


In [4]:
import polars as pl

print(pl.__version__)
print(dir(pl))


1.35.2
['Any', 'Array', 'BasePartitionContext', 'Binary', 'Boolean', 'Catalog', 'Categorical', 'Categories', 'CompatLevel', 'Config', 'CredentialProvider', 'CredentialProviderAWS', 'CredentialProviderAzure', 'CredentialProviderFunction', 'CredentialProviderFunctionReturn', 'CredentialProviderGCP', 'DataFrame', 'DataType', 'DataTypeExpr', 'Date', 'Datetime', 'Decimal', 'Duration', 'Enum', 'Expr', 'Field', 'Float32', 'Float64', 'GPUEngine', 'Int128', 'Int16', 'Int32', 'Int64', 'Int8', 'KeyedPartition', 'KeyedPartitionContext', 'LazyFrame', 'List', 'Null', 'Object', 'PartitionByKey', 'PartitionMaxSize', 'PartitionParted', 'QueryOptFlags', 'SQLContext', 'ScanCastOptions', 'Schema', 'Series', 'String', 'StringCache', 'Struct', 'TYPE_CHECKING', 'Time', 'UInt128', 'UInt16', 'UInt32', 'UInt64', 'UInt8', 'Unknown', 'Utf8', '__all__', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__register_startup_dep

In [11]:
import polars as pl
import math

def restaurants_autour(df, lat0, lon0, rayon_m):
    R = 6371000.0

    lat0_rad = math.radians(lat0)
    lon0_rad = math.radians(lon0)

    # Réduit les données gérées
    lat_margin = 0.1   # ~ 11 km
    lat_margin = lat_margin * (rayon_m/1000) # Raisonnable ici (ex: pour 500m on prends une marge de 0.05 de lat soit 5.5km
    lon_margin = 0.1   # ~ 7.5 km
    lon_margin = lon_margin * (rayon_m/1000) # Raisonnable ici (ex: pour 500m on prends une marge de 0.05 de lat soit 3.7km

    return (
        df
        .filter(pl.col("amenity") == "restaurant")

        .filter(
            (pl.col("@lat").is_between(lat0 - lat_margin, lat0 + lat_margin)) &
            (pl.col("@lon").is_between(lon0 - lon_margin, lon0 + lon_margin))
        )

        # radians
        .with_columns([
            pl.col("@lat").cast(pl.Float64).radians().alias("lat_rad"),
            pl.col("@lon").cast(pl.Float64).radians().alias("lon_rad"),
        ])

        # deltas
        .with_columns([
            (pl.col("lat_rad") - lat0_rad).alias("dlat"),
            (pl.col("lon_rad") - lon0_rad).alias("dlon"),
        ])

        # h = haversine
        .with_columns([
            (
                (pl.col("dlat") / 2).sin() ** 2
                + math.cos(lat0_rad)
                * pl.col("lat_rad").cos()
                * (pl.col("dlon") / 2).sin() ** 2
            ).alias("h")
        ])

        # Approximation arcsin(x)
        # arcsin(x) ≈ x * (1 + 0.5 * x^2) / sqrt(1 - x^2)
        .with_columns([
            pl.col("h").pow(0.5).alias("sqrt_h"),
            (1 - pl.col("h")).pow(0.5).alias("sqrt_1mh"),
        ])

        .with_columns([
            (
                2 * R *
                (
                    pl.col("sqrt_h") * (1 + 0.5 * pl.col("h"))
                    / pl.col("sqrt_1mh")
                )
            ).alias("distance_m")
        ])

        .filter(pl.col("distance_m") <= rayon_m)
        .sort("distance_m")

        .select([
            "@id", "@lat", "@lon", "name", "local_name",
            "amenity", "distance_m"
        ])
    )


In [12]:
restaurant_autour_mairie_quimper = restaurants_autour(oms_df, lat0, lon0, 200)

print(f"Nb restaurants: {len(restaurant_autour_mairie_quimper)}")

restaurant_autour_mairie_quimper

Nb restaurants: 31


@id,@lat,@lon,name,local_name,amenity,distance_m
i64,f64,f64,str,str,str,f64
1411218821,47.996147,-4.101604,"""La Cabane à crêpes""",null,"""restaurant""",32.237115
2123572471,47.995886,-4.101395,"""Copper J""",null,"""restaurant""",58.621375
2028117630,47.995908,-4.101306,"""La Pépinière""",null,"""restaurant""",62.77878
9657124470,47.9958,-4.101312,"""Coq Rock""",null,"""restaurant""",69.331614
432655229,47.996682,-4.102704,"""Crêperie Amann Rik""",null,"""restaurant""",73.578074
…,…,…,…,…,…,…
11891810642,47.99501,-4.103519,"""Ty Loulic""",null,"""restaurant""",172.336946
471542772,47.997398,-4.100504,"""Les Douves""",null,"""restaurant""",175.398284
6548518328,47.996452,-4.104393,"""Eskemm""",null,"""restaurant""",178.008925


In [16]:
restaurant_autour_mairie_quimper.to_pandas().to_csv("oms_export_restaurant_200m_mairie.csv")
# pl.from_dataframe(restaurant_autour_mairie_quimper).to_csv("oms_export_restaurant_200m_mairie.csv")